In [3]:
import pandas as pd

import pandas as pd
import glob
cornbelters_files = glob.glob("../CornBeltersData/*.csv")

all_files = cornbelters_files 

# Read and concatenate all CSVs into one DataFrame
xba_data_list = [pd.read_csv(f) for f in all_files]
df = pd.concat(xba_data_list, ignore_index=True)

#Change this per team
# Filter for Normal CornBelters batters
df = df[df['BatterTeam'] == 'Normal cornbelters']

# Function to compute stats for a given pitcher handedness
def batter_stats_vs_pitcher(df, pitcher_hand):
    filtered = df[df['PitcherThrows'] == pitcher_hand].dropna(subset=['PlayResult'])
    # Plate appearances: count of unique PitchNo per batter (or use PAofInning if available)

    # Exclude rows that are not official at bats

    # Homeruns
    hr = filtered[filtered['PlayResult'] == 'HomeRun'].groupby(['Batter']).size().rename('homeruns')
    # Triples
    triples = filtered[filtered['PlayResult'] == 'Triple'].groupby(['Batter']).size().rename('triples')
    # Doubles
    doubles = filtered[filtered['PlayResult'] == 'Double'].groupby(['Batter']).size().rename('doubles')
    # Singles (PlayResult == 'Single')
    singles = filtered[filtered['PlayResult'] == 'Single'].groupby(['Batter']).size().rename('singles')
    # Walks (PitchCall == 'Walk' or 'IntentionalWalk')
    walks = filtered[filtered['KorBB'].isin(['Walk'])].groupby(['Batter']).size().rename('walks')
    # Outs (PlayResult == 'Out' or similar)
    outs = filtered[filtered['PlayResult'].isin(['StrikeoutLooking','StrikeoutSwinging','Out'])].groupby(['Batter']).size().rename('outs')

    # Assuming you have a DataFrame 'stats' with these columns for each batter
    
    # Combine all stats
    stats = pd.concat([hr, triples, doubles, singles, walks, outs], axis=1).fillna(0).reset_index()
    stats['at_bats'] = (
        stats['homeruns'] +
        stats['triples'] +
        stats['doubles'] +
        stats['singles'] +
        stats['outs']
    )
    # Define the player-to-position mapping
    cornbelters_player_position = {
        'Jackson Smith': 'if',
        'Jackson Stanek': 'if',
        'Tyler Thompson': 'if',
        'Nolan Mccrossin' : 'if',
        'Shea Zbrozek': 'if',
        'Jack Novak': 'of',
        'Kannan Kleine': 'c',
        'Caleb Royer': 'if',
        'Charlie Graham': 'c',
        'Cole Freeman': 'of',
        'Michael Kuska': 'of',
        'Ryan Colucci': 'if',
        'Zach Zychowski': 'if'
    }
    bluecaps_player_position = {
        'Zach Bava': 'c',
        'Braxton Waller': 'c',
        'Nolan Hunter': 'c',
        'Jacob Trusner': 'if',
        'Trey Schilling': 'if',
        'Michael Lucas': 'if',
        'Jackson Natanek': 'if',
        'Cameron Anderson': 'if',
        'Jake Morrill': 'of',
        'Grayson Schnierle': 'of',
        'Riley Hendren': 'of',
        'Cal Darling': 'of',
        'Trey Blanchette': 'of',
        'Chase Peterson': 'of',
        'Andrew Schweigert': 'of'
    }
    bobcats_player_position = {
    'Ryan Theile': 'c',
    'Zach Pomatto': 'c',
    'Jake Zimmerman': 'c',
    'Kyler Lapp': 'if',
    'Nolan Galla': 'if',
    'Colin Karr': 'if',
    'Max Michalak': 'if',
    'Will Strehl': 'if',
    'Jake Folmar': 'if',
    'Luke Gallet': 'if',
    'Brady Veselack': 'if',
    'Kaileb Hackman': 'if',
    'Zach Spidle': 'of',
    'Max Wille': 'of',
    'Cody Delfavero': 'of',
    'Nick Guidici': 'of',
    'Zayn Giles': 'of'
    }
    groundsloths_player_position = {
    'Will Vogel': 'c',
    'Drew Wichtowski': 'c',
    'Nolan Bowles': 'c',
    'Sam Descarpentrie': 'c',
    'Lucas Rink': 'if',
    'Gabe Peters': 'if',
    'Cameron Heinold': 'if',
    'Clint Goodman': 'if',
    'Cameron Steinbaugh': 'if',
    'Brayden Zilis': 'if',
    'Ty Cribbett': 'if',
    'Bennett Summers': 'if',
    'Mitchell Wilson': 'if',  # LHP/OF, but not a clear IF, remove if you want only pure IF
    'Adan Nieves': 'if',      # OF/INF/RHP, mapped to IF
    'Bennett Summers': 'if',  # INF/OF, mapped to IF
    'Tyler Dowling': 'of',    # LHP/OF, mapped to OF
    'Adan Nieves': 'of',      # OF/INF/RHP, mapped to OF
    'Gavin Bailey': 'of',
    'Kaden Griswold': 'of',
    'Brandon Schultz': 'of',
    'Trey Thompson': 'of', 
    'Luke Mcclure' : 'of'   # OF/LHP, mapped to OF
    }
    merchants_player_position = {
    'Nolan Vanduzer': 'if',      # RHP/3B/1B
    'Blake Willan': 'if',        # RHP/CINF (assuming CINF = corner infielder)
    'Tyler Pippin': 'if',        # INF
    'Cole Yates': 'if',          # INF
    'Kaiden Maurer': 'if',       # INF
    'Brayden Elliott': 'if',     # RHP/SS
    'Noah Dill': 'if',           # 3B
    'Brooks Neuhoff': 'if',      # INF/OF (first match is IF)
    'Jaxson Grubbs': 'if',       # 1B
    'Taylor Vorhees': 'if',      # INF
    'Lawson Alwan': 'c',         # Catcher
    'Teagan Disharoon': 'c',     # RHP/Catcher
    'Gavin Kleist': 'c',         # Catcher
    'Jake Parpet': 'c',          # Catcher
    'Braden Mehn': 'of',         # LHP/OF
    'Thomas Mickels': 'of',      # RHP/OF
    'Lawson Shickel': 'of',      # RHP/OF/1B (first match is OF after RHP)
    'Brady Hart': 'of',          # OF/3B
    'Jackson Newton': 'of',      # CF
    'Deegan Aeilts': 'of',       # OF
    'Jacob Blunck': 'of',        # OF
}
    # Add a new 'position' column to your DataFrame based on the mapping
    stats['position'] = stats['Batter'].map(cornbelters_player_position)
    stats['order'] = range(1, len(stats) + 1)
    stats['player_name'] = stats['Batter']
    stats = stats[['order','position', 'player_name', 'homeruns', 'triples', 'doubles', 'singles', 'walks', 'outs', 'at_bats']]
    cols = ['homeruns', 'triples', 'doubles', 'singles', 'walks', 'outs', 'at_bats']
    stats[cols] = stats[cols].fillna(0).astype(int)
    return stats

# For right-handed pitchers
stats_vs_right = batter_stats_vs_pitcher(df, 'Right')
# For left-handed pitchers
stats_vs_left = batter_stats_vs_pitcher(df, 'Left')

# Example: print both DataFrames
print("Vs Right-handed Pitchers:")
print(stats_vs_right)
print("\nVs Left-handed Pitchers:")
print(stats_vs_left)
stats_vs_left.to_csv('stats_vs_left.csv', index=False)
stats_vs_right.to_csv('stats_vs_right.csv', index=False)

Vs Right-handed Pitchers:
    order position      player_name  homeruns  triples  doubles  singles  \
0       1       if      Caleb Royer         1        0        0        9   
1       2       of     Cole Freeman         1        0        0        3   
2       3       if    Jackson Smith         1        0        1        8   
3       4      NaN   Tony Konopiots         1        0        1        1   
4       5       if   Tyler Thompson         0        1        0       10   
5       6       of       Jack Novak         0        0        4        6   
6       7       if   Jackson Stanek         0        0        3        6   
7       8       if  Nolan Mccrossin         0        0        1        7   
8       9       if     Ryan Colucci         0        0        1        3   
9      10       if   Zach Zychowski         0        0        1        5   
10     11      NaN   Austin Weigand         0        0        0        1   
11     12        c   Charlie Graham         0        0        

In [1]:

import pandas as pd

# Assume df is your original DataFrame
df = pd.read_csv("2025.csv")
df_new = pd.DataFrame()
df_new['order'] = df['#'].astype(int)
df_new['position'] = df['Pos']
df_new['player_name'] = df['Name']
df_new['homeruns'] = df['hr'].astype(int)
df_new['triples'] = df['3b'].astype(int)
df_new['doubles'] = df['2b'].astype(int)
df_new['singles'] = (df['h'] - df['2b'] - df['3b'] - df['hr']).astype(int)
df_new['walks'] = df['bb'].astype(int)
df_new['outs'] = (df['ab'] - df['h']).astype(int)
df_new['at_bats'] = df['ab'].astype(int)



ValueError: invalid literal for int() with base 10: "'-"